In [ ]:
from google.colab import drive
# drive.mount('/content/drive/MyDrive/IITP/sohyun/creditcard_prediction/data')
drive.mount('/content/drive')

%cd drive/MyDrive/IITP/sohyun/TimeSeriesAnomaly/data/modify

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1j1N0u5t0l99N_wfSd5UZvnhugzn5g_NC/TimeSeriesAnomaly/data/modify


In [ ]:
# !pip install wandb -qqq
# import wandb
# wandb.login()

In [ ]:
import matplotlib.pyplot as plt
import easydict
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import random
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
import time
import math

## Data 

In [ ]:
#---# LOAD npy file #---#
Fu_20_normal = np.load('Fu_20_normal.npy')
Fu_21_normal = np.load('Fu_21_normal.npy')
Fu_21_abnormal = np.load('Fu_21_abnormal.npy')
Fu_22_normal = np.load('Fu_22_normal.npy')
Fu_22_abnormal = np.load('Fu_22_abnormal.npy')

Fu_20_normal_10 = np.load('Fu_20_normal_10.npy')
Fu_21_normal_10 = np.load('Fu_21_normal_10.npy')
Fu_21_abnormal_10 = np.load('Fu_21_abnormal_10.npy')
Fu_22_normal_10 = np.load('Fu_22_normal_10.npy')
Fu_22_abnormal_10 = np.load('Fu_22_abnormal_10.npy')

# import sys
# np.set_printoptions(threshold=sys.maxsize) # print all

#---# 확인용 #---#
# plt.figure(figsize=(30,5))
# plt.plot(Fu_22_abnormal_10)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

class MyDataset(Dataset):
  def __init__(self, data):
    self.data = np.array(self.sliding_window(data, config.window_size, config.stride))
    self.data = self.data.reshape(-1, config.window_size)
    
  def __getitem__(self, index):
    self.x = self.data[index]
    return (index, torch.Tensor(self.x))
      
  def __len__(self):
    return len(self.data)

  #---# Window #---#
  def sliding_window(self, arr, window_size, stride):
    start_pt = 0
    total_data = []
    while(True) :
      if len(arr) < (start_pt + window_size) : break
      data = arr[start_pt:start_pt+window_size]
      start_pt += stride
      total_data.append(data)
    return total_data


def drawing(pred, x) :
  #---# Drawing - 22 #---#
  plt.figure(figsize=(30,5))
  plt.plot(x, markersize=1)
  plt.plot(pred, marker='.', markersize=2, color='r', linestyle='None')

  '''
  for i in range(len(pred)):
    if pred[i] == 1:
      # plt.vlines(i * config.window_size, -1, 4, colors="y")
      # plt.vlines((i+1) * config.window_size, -1, 4, colors="y")
      before = i * config.window_size
      after = (i+1) * config.window_size

      r = np.linspace(before, after)
      plt.fill_between(r, -1, 4, color = "yellow", alpha = 0.5)
  '''

  #---# 실제 anomaly 값 구간 #---#
  a = np.linspace(62200, 65300)
  # plt.fill_between(a, 0, 2000, color='green', alpha=0.3)
  plt.fill_between(a, -1, 4, color='green', alpha=0.5)
  b = np.linspace(95600, 99300)
  # plt.fill_between(b, 0, 2000, color='green', alpha=0.5)
  plt.fill_between(b, -1, 4, color='green', alpha=0.5)
  c = np.linspace(148400, 152300)
  # plt.fill_between(c, 0, 2000, color='green', alpha=0.5)
  plt.fill_between(c, -1, 4, color='green', alpha=0.5)

  plt.show()
  plt.clf()

In [ ]:
config = easydict.EasyDict({
    "num_epochs" : 500, #500
    "batch_size" : 16, #16 
    "mode" : 'train',
    # "mode" : "test",
    "lr" : 1e-3, 
    "wd" : None,
    "window_size" : 1000,
    "stride" : 250,
    "threshold" : 0.05 # 0.3이나 0.2로 하기
})

tm = time.localtime(time.time())
string = time.strftime('%Y%m%d_%H%M%S', tm)

# wandb.init(project="Anomaly-Oil", entity="sohyun", name=string, magic=True)

train = pd.DataFrame(Fu_22_normal_10, columns=['Fu'])
train = pd.concat([train, pd.DataFrame(Fu_21_normal_10, columns=['Fu']), pd.DataFrame(Fu_20_normal_10, columns=['Fu'])], axis=0)
test = pd.DataFrame(Fu_22_abnormal_10, columns=['Fu'])
test_len = len(test)
total = pd.DataFrame(pd.concat([train, test], axis=0))

# train_scale = train['Fu'].values
# test_scale = test['Fu'].values
# total_scale = total['Fu'].values

#---# Noramlize #---#
scaler = StandardScaler()
total_scale = scaler.fit_transform(total); total_scale = pd.DataFrame(total_scale, columns=['Fu'])['Fu'].values.tolist() # total 먼저 해놓고 transform
train_scale = scaler.transform(train); train_scale = pd.DataFrame(train_scale, columns=['Fu'])['Fu'].values.tolist()
test_scale = scaler.transform(test); test_scale = pd.DataFrame(test_scale, columns=['Fu'])['Fu'].values.tolist()

# scaler = MinMaxScaler()
# total_scale = scaler.fit_transform(total); total_scale = pd.DataFrame(total_scale, columns=['Fu'])['Fu'].values.tolist()
# test_scale = scaler.transform(test); test_scale = pd.DataFrame(test_scale, columns=['Fu'])['Fu'].values.tolist()

#---# Setting train data #---# 나머지 끊어주어야 함
train_dataset1 = MyDataset(data=train_scale[:len(Fu_22_normal_10)])
train_dataset2 = MyDataset(data=train_scale[len(Fu_22_normal_10):(len(Fu_22_normal_10)+len(Fu_21_normal_10))])
train_dataset3 = MyDataset(data=train_scale[(len(Fu_22_normal_10)+len(Fu_21_normal_10)):])
train_dataset = np.concatenate([train_dataset1.data, train_dataset2.data, train_dataset3.data], axis = 0)

train_loader = DataLoader(train_dataset, batch_size=config.batch_size, shuffle=True, drop_last=True)
test_dataset = MyDataset(data=test_scale)
test_loader = DataLoader(test_dataset, batch_size=config.batch_size, shuffle=False)


# optimizer = torch.optim.Adam(params = model.parameters(), lr = config.lr) # lr = config.lr
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=10, threshold_mode='abs', min_lr=1e-8, verbose=True)


In [ ]:
!pip install orion-ml -qqq

In [ ]:
####################
#---# 예제 데이터 #---#
####################
# from orion.data import load_signal

# train_data = load_signal('S-1-train')
# # timestamp2 = np.arange(0, len(train_data)/100, 0.01)
# # train_data['timestamp'] = timestamp2

# from orion import Orion
# hyperparameters = {
#     'keras.Sequential.LSTMTimeSeriesRegressor#1' : {
#         'epochs' : 3,
#         'verbose' : True
#     }
# }
# orion = Orion(
#     pipeline = 'lstm_dynamic_threshold',
#     hyperparameters = hyperparameters
# )
# orion.fit(train_data)
# new_data = load_signal('S-1-new')
# anomalies = orion.detect(new_data)
# print(anomalies)

In [ ]:
from pandas.tseries.offsets import Second
from datetime import date, timedelta
from orion import Orion
from datetime import datetime

def make_timestamp(date, data_len) :
  # start_date = datetime.strptime('201802210000', '%Y%m%d%H%M')
  start_date = datetime.strptime(date, '%Y%m%d%H%M')
  timestamp = time.mktime(start_date.timetuple())
  dates = []; dates.append(timestamp)

  for i in range(data_len-1) :
    new_start_date = start_date + timedelta(minutes=10)
    timestamp = time.mktime(new_start_date.timetuple())
    dates.append(timestamp)
    start_date = new_start_date
  
  return dates

dates = make_timestamp('201802210000', len(train_scale))
traindata = pd.DataFrame({"timestamp":dates, "value":train_scale})
dates_test = make_timestamp('201802211200', len(test_scale))
testdata = pd.DataFrame({"timestamp":dates_test, "value":test_scale})

hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
      "target_column": 0,
      "window_size": 1000,
      "step_size": 250
    },
    "numpy.reshape#1": {
        "newshape": [
            -1,
            1000
        ]
    },
    "statsmodels.tsa.arima_model.Arima#1": {
          "steps": 2,
          "p" : 2
    },
    "orion.primitives.timeseries_anomalies.find_anomalies#1": {
          "fixed_threshold": True
    }
}      

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA', FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA', FutureWarning)

orion = Orion(
    pipeline = [
      "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences",
      "numpy.reshape",
      "statsmodels.tsa.arima_model.Arima",
      "orion.primitives.timeseries_errors.regression_errors",
      "orion.primitives.timeseries_anomalies.find_anomalies"
    ] , #'lstm_dynamic_threshold'
    hyperparameters = hyperparameters
)

print("============ FIT ============")
orion.fit(np.array(train_scale).reshape(-1,1), index=np.array(range(len(train_scale))).reshape(-1,1))

print("============ DETECT ============")
# print(orion.detect(testdata))
print(orion.detect(test_scale).reshape(-1,1), index=np.array(range(len(test_scale))).reshape(-1,1))
scores = orion.evaluate(testdata, ground_truth, fit=True)

In [ ]:
from pandas.tseries.offsets import Second
from datetime import date, timedelta
from orion import Orion
from datetime import datetime

def make_timestamp(date, data_len) :
  # start_date = datetime.strptime('201802210000', '%Y%m%d%H%M')
  start_date = datetime.strptime(date, '%Y%m%d%H%M')
  timestamp = time.mktime(start_date.timetuple())
  dates = []; dates.append(timestamp)

  for i in range(data_len-1) :
    new_start_date = start_date + timedelta(minutes=10)
    timestamp = time.mktime(new_start_date.timetuple())
    dates.append(timestamp)
    start_date = new_start_date
  
  return dates

dates = make_timestamp('201802210000', len(train_scale))
traindata = pd.DataFrame({"timestamp":dates, "value":train_scale})
dates_test = make_timestamp('201802211200', len(test_scale))
testdata = pd.DataFrame({"timestamp":dates_test, "value":test_scale})

hyperparameters = {
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences#1": {
      "target_column": 0,
      "window_size": 1000,
      "step_size": 250
    },
    "orion.primitives.timeseries_preprocessing.slice_array_by_dims#1" : {
      "target_index" : 0,
      "axis" : 2
    },
    "keras.Sequential.LSTMSeq2Seq#1" : {
        "epochs" : 30,
        "window_size" : 1000,
        "input_size" : [1000,1],
        # "target_size" : [1000,1]
    },
    # "orion.primitives.timeseries_anomalies.find_anomalies#1" : {
    #     "window_size_portion" : 0.3,
    #     "fixed_threshold" : True
    # }
}      


orion = Orion(
  pipeline = [
    "mlprimitives.custom.timeseries_preprocessing.rolling_window_sequences",
    "orion.primitives.timeseries_preprocessing.slice_array_by_dims",
    "keras.Sequential.LSTMSeq2Seq",
    # "orion.primitives.timeseries_errors.reconstruction_errors",
    # "orion.primitives.timeseries_anomalies.find_anomalies"
  ], #'lstm_dynamic_threshold'
  hyperparameters = hyperparameters
)

print("============ FIT ============")
orion.fit(np.array(train_scale).reshape(-1,1), index=np.array(range(len(train_scale))).reshape(-1,1))

print("============ DETECT ============")
# print(orion.detect(testdata))
print(orion.detect(test_scale).reshape(-1,1), index=np.array(range(len(test_scale))).reshape(-1,1))